In [1]:
import sys
import pdfplumber
print(f"현재 파이썬 경로: {sys.executable}")
print("✅ pdfplumber 로드 성공!")

현재 파이썬 경로: /home/spai0630/workspace/venv/bin/python
✅ pdfplumber 로드 성공!


In [2]:
import os
import pandas as pd

# 경로 다시 확인
BASE_DIR = "/home/spai0630/workspace/data/original_data"
FILES_DIR = os.path.join(BASE_DIR, "files")
META_PATH = os.path.join(BASE_DIR, "data_list.csv")

# 1. 폴더에 파일이 진짜 있는지 확인
if os.path.exists(FILES_DIR):
    files = os.listdir(FILES_DIR)
    print(f"✅ 폴더 감지 성공! 파일 개수: {len(files)}개")
    print(f"샘플 파일명: {files[:3]}")
else:
    print("❌ 폴더 경로가 잘못되었습니다. 경로를 다시 확인하세요!")

# 2. CSV 파일 로드 및 파일명 매칭 테스트
df = pd.read_csv(META_PATH)
csv_name = df['파일명'].iloc[0]
print(f"📊 CSV 첫 번째 파일명: '{csv_name}'")
print(f"🔍 매칭 결과: {csv_name in files}")

✅ 폴더 감지 성공! 파일 개수: 101개
샘플 파일명: ['사단법인아시아물위원회사무국_우즈벡-키르기즈스탄 기후변화대응 스.hwp', '축산물품질평가원_축산물이력관리시스템 개선(정보화 사업).hwp', '한국수출입은행_(긴급) 모잠비크 마푸토 지능형교통시스템(ITS) 구축사업.hwp']
📊 CSV 첫 번째 파일명: '한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp'
🔍 매칭 결과: False


pdfplumner는 pdf 전용 라이브러리 라서 한글 파일을 못읽음 그래서 확장자 불일치 존재 
그래서 pdf로 모든 한글파일을 변환해서 기존 로직을 사용한다거나 
한글 전용 파서를 사용하는 방법 pyhwpx 나 olefile 같은 라이브러리 추가 설치 
혹은 pdf만 갖고 먼저 rag 성능 테스트 

In [3]:
import os
import pandas as pd

# 1. 확장자를 제거한 파일 리스트 만들기
file_list = os.listdir(FILES_DIR)
file_names_only = [os.path.splitext(f)[0] for f in file_list]

# 2. CSV 데이터 확인
df = pd.read_csv(META_PATH)

print("📋 [매칭 분석 결과]")
for i in range(5):
    csv_name = df['파일명'].iloc[i]
    csv_name_no_ext = os.path.splitext(csv_name)[0].strip()
    
    # 리스트에 있는지 확인
    match = csv_name_no_ext in file_names_only
    print(f"CSV: {csv_name_no_ext[:20]}... | 매칭 여부: {'✅' if match else '❌'}")

# 3. HWP 파일 개수 확인
hwp_count = len([f for f in file_list if f.endswith('.hwp')])
pdf_count = len([f for f in file_list if f.endswith('.pdf')])
print(f"\n📊 현재 폴더 구성 -> HWP: {hwp_count}개, PDF: {pdf_count}개")

📋 [매칭 분석 결과]
CSV: 한영대학_한영대학교 특성화 맞춤형 교... | 매칭 여부: ❌
CSV: 한국연구재단_2024년 대학산학협력활... | 매칭 여부: ❌
CSV: 한국생산기술연구원_EIP3.0 고압가... | 매칭 여부: ❌
CSV: 인천광역시_도시계획위원회 통합관리시스... | 매칭 여부: ❌
CSV: 경상북도 봉화군_봉화군 재난통합관리시... | 매칭 여부: ❌

📊 현재 폴더 구성 -> HWP: 96개, PDF: 4개


pdf 갯수가 현저히 적기 때문에 pdf로 변환해서 사용하는 건 너무 비효율적이므로 한글 전용 라이브러리를 인스톨해서 사용해보기로 함 

In [4]:
pip install olefile

Note: you may need to restart the kernel to use updated packages.


In [5]:
import olefile
import zlib

def extract_hwp_text(hwp_path):
    f = olefile.OleFileIO(hwp_path)
    dirs = f.listdir()

    # HWP 내 본문 스트림(BodyText) 찾기
    bodytext = [d for d in dirs if d[0].startswith('BodyText')]
    full_text = ""

    for section in bodytext:
        data = f.openstream(section).read()
        # 압축 해제 (HWP는 기본적으로 zlib 압축됨)
        try:
            decompressed = zlib.decompress(data, -15)
            full_text += decompressed.decode('utf-16', errors='ignore')
        except:
            pass
            
    return full_text

# 테스트 실행
test_hwp = os.path.join(FILES_DIR, file_list[0]) # 아까 찾은 HWP 파일
if test_hwp.endswith('.hwp'):
    print(f"📄 '{test_hwp}' 추출 시도 중...")
    print(extract_hwp_text(test_hwp)[:500]) # 앞부분 500자만 확인

📄 '/home/spai0630/workspace/data/original_data/files/사단법인아시아물위원회사무국_우즈벡-키르기즈스탄 기후변화대응 스.hwp' 추출 시도 중...
ф  хɀ    Ϩ Ϩ ͒ ɘ   룼  ч˰捤獥  Ѯ  ὀ              䤀耈蠂èň搀─ሀ鬀鬀鬀    䨀쀈   ⤀Ā＀￿勿㜃ᬂā 䨀쀈   ⤀Ā刀㜃Ā 䬀Ā 褀褅褅褅܅䬀Ā 褀褅褅褅ą䬀Ā 褀褅褅褅ą䜀搁潬ѣ    䈀耀ā   ᠀ Ā Ā  䐀耄  ࠀ 䔀䀄 䀀刀堀  ﰀ¸ ؀䈀耀ā   ᠀ Ā Ā  䐀耄  ࠀ 䔀䀄 耀刀堀  ﰀ¸ ؀䈀耀ᄁ   ᠀ Ȁ Ā  䌀 ᔂ搀杨p   ᔀ଀ 獯g   ଀ഀ䐀 ࠀ ࠀ ऀ 䔀䀄 쀀##耀堀  ﰀ¸ ؀䜀耄搀杨⁰ 䜀 獯ᅧ橃   ⬀#Ā     頀관Z  䰀 渍捯渤捯$    Ā䐀¨#⬀# Ā ᔀMĀ   ?              ?   ଀搿ꠗ?              ?      ?              ?   Ȁ挀灩挤牥餤관䰚挒灩蠤￸뫿￸ǿĀ퀀¶1縀§1  $ᔀMȀ   ?      鷠À      


olefile 은 파일이 개같이 깨졌음.
그래서 다른 팀원들이 활용해봤다던 hwp5python 라이브러리 활용해보겠습니다.


In [6]:
pip install hwp5python

ERROR: Could not find a version that satisfies the requirement hwp5python (from versions: none)
ERROR: No matching distribution found for hwp5python
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install hwp5

ERROR: Could not find a version that satisfies the requirement hwp5 (from versions: none)
ERROR: No matching distribution found for hwp5
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pyhwpx

INFO: pip is looking at multiple versions of pyhwpx to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of pyhwpx to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "/home/spai0630/workspace/venv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
        File "/h

윈도우 노트북이 아니라서? 한글 파일 로드 및 라이브러리 설치 실패 

In [10]:
import pdfplumber
import os
import pandas as pd
from tqdm import tqdm

# 1. 경로 설정 (정확한 spai0630 계정 경로)
BASE_DIR = "/home/spai0630/workspace/data/original_data"
FILES_DIR = os.path.join(BASE_DIR, "files")
META_PATH = os.path.join(BASE_DIR, "data_list.csv")

# 2. 메타데이터 로드 (여기서 df_meta를 정의합니다)
if os.path.exists(META_PATH):
    df_meta = pd.read_csv(META_PATH)
    print(f"📊 CSV 로드 성공: {len(df_meta)}개의 목록이 있습니다.")
else:
    print("❌ CSV 파일을 찾을 수 없습니다. 경로를 확인하세요.")

# 3. PDF 파일만 골라내서 처리
results = []
if os.path.exists(FILES_DIR):
    pdf_files = [f for f in os.listdir(FILES_DIR) if f.lower().endswith('.pdf')]
    print(f"✅ 발견된 PDF 개수: {len(pdf_files)}개")

    for index, row in tqdm(df_meta.iterrows(), total=len(df_meta)):
        f_name = str(row['파일명']).strip() # 파일명을 문자열로 변환 후 공백 제거
        if f_name in pdf_files:
            f_path = os.path.join(FILES_DIR, f_name)
            try:
                with pdfplumber.open(f_path) as pdf:
                    # 모든 페이지 텍스트 추출
                    text = " ".join([p.extract_text() for p in pdf.pages if p.extract_text()])
                    results.append({
                        "사업명": row.get('사업명', 'N/A'),
                        "발주기관": row.get('발주기관', 'N/A'),
                        "본문": text,
                        "파일명": f_name
                    })
            except Exception as e:
                print(f"❌ {f_name} 처리 중 에러: {e}")

    # 4. 결과 저장
    output_path = "/home/spai0630/workspace/data/processed_pdf_only.jsonl"
    df_result = pd.DataFrame(results)
    df_result.to_json(output_path, orient='records', force_ascii=False, lines=True)
    print(f"🚀 최종 완료: {len(results)}건의 PDF가 통합되었습니다!")
else:
    print("❌ files 폴더를 찾을 수 없습니다.")

📊 CSV 로드 성공: 100개의 목록이 있습니다.
✅ 발견된 PDF 개수: 4개


100%|██████████| 100/100 [00:00<00:00, 20843.33it/s]

🚀 최종 완료: 0건의 PDF가 통합되었습니다!


사소한 공백이나 파일명의 차이 때문에 매칭이 이뤄지지 않아서 실제 파일들은 서칭이 되었지만 통합이 이뤄지지 않음 

In [11]:
import os
import pandas as pd

# 1. 실제 폴더에 있는 PDF 파일명 리스트 출력
pdf_files = [f for f in os.listdir(FILES_DIR) if f.lower().endswith('.pdf')]
print(f"📁 서버에 있는 실제 PDF (4개): {pdf_files}")

# 2. CSV에서 PDF라고 주장하는 파일명들 출력
df_meta = pd.read_csv(META_PATH)
csv_pdf_list = [f for f in df_meta['파일명'].tolist() if str(f).lower().endswith('.pdf')]
print(f"📊 CSV에 적힌 PDF 목록 (일부): {csv_pdf_list[:5]}")

# 3. 왜 안 맞는지 하나만 정밀 비교
if pdf_files and csv_pdf_list:
    print("\n🧐 정밀 비교 테스트:")
    print(f"실제파일: '{pdf_files[0]}'")
    print(f"CSV파일 : '{csv_pdf_list[0]}'")
    print(f"일치여부: {pdf_files[0] == csv_pdf_list[0]}")
    

📁 서버에 있는 실제 PDF (4개): ['고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf', '기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf', '서울시립대학교_[사전공개] 학업성취도 다차원 종단분석 통합시스템 1차.pdf', '서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf']
📊 CSV에 적힌 PDF 목록 (일부): ['고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf', '서울시립대학교_[사전공개] 학업성취도 다차원 종단분석 통합시스템 1차.pdf', '서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf', '기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf']

🧐 정밀 비교 테스트:
실제파일: '고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf'
CSV파일 : '고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf'
일치여부: False


In [12]:
for index, row in tqdm(df_meta.iterrows(), total=len(df_meta)):
    csv_f_name = str(row['파일명']).strip().lower() # 소문자로 통일 및 공백 제거
    
    # 실제 폴더의 파일들 중 CSV 파일명을 포함하는 것이 있는지 확인
    matched_file = None
    for real_f in pdf_files:
        if csv_f_name in real_f.lower() or real_f.lower() in csv_f_name:
            matched_file = real_f
            break
            
    if matched_file:
        f_path = os.path.join(FILES_DIR, matched_file)
        # (이후 pdfplumber 로직 동일...)

100%|██████████| 100/100 [00:00<00:00, 14829.77it/s]


In [13]:
import unicodedata
import os
import pdfplumber
import pandas as pd
from tqdm import tqdm

# 1. 파일 목록을 가져올 때 'NFC'로 통일해서 읽기
raw_files = os.listdir(FILES_DIR)
pdf_files = [unicodedata.normalize('NFC', f) for f in raw_files if f.lower().endswith('.pdf')]
print(f"✅ 정규화 완료된 PDF 목록: {pdf_files}")

# 2. 매칭 로직에도 정규화 적용
results = []
for index, row in tqdm(df_meta.iterrows(), total=len(df_meta)):
    # CSV의 파일명도 NFC로 정규화
    csv_f_name = unicodedata.normalize('NFC', str(row['파일명']).strip())
    
    if csv_f_name in pdf_files:
        # 실제 서버 파일 시스템에 접근할 때는 원래의 이름을 찾아야 하므로 매칭 수행
        target_file = None
        for original_f in raw_files:
            if unicodedata.normalize('NFC', original_f) == csv_f_name:
                target_file = original_f
                break
        
        if target_file:
            f_path = os.path.join(FILES_DIR, target_file)
            try:
                with pdfplumber.open(f_path) as pdf:
                    text = " ".join([p.extract_text() for p in pdf.pages if p.extract_text()])
                    results.append({
                        "사업명": row.get('사업명', 'N/A'),
                        "본문": text,
                        "파일명": csv_f_name
                    })
            except Exception as e:
                print(f"❌ {csv_f_name} 에러: {e}")

# 3. 결과 저장
output_path = "/home/spai0630/workspace/data/processed_pdf_only.jsonl"
pd.DataFrame(results).to_json(output_path, orient='records', force_ascii=False, lines=True)
print(f"🚀 드디어 성공! {len(results)}건의 PDF가 통합되었습니다.")

✅ 정규화 완료된 PDF 목록: ['고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf', '기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf', '서울시립대학교_[사전공개] 학업성취도 다차원 종단분석 통합시스템 1차.pdf', '서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf']


100%|██████████| 100/100 [02:17<00:00,  1.38s/it]

🚀 드디어 성공! 4건의 PDF가 통합되었습니다.


In [14]:
import json

# 1. 전처리된 데이터 로드
input_path = "/home/spai0630/workspace/data/processed_pdf_only.jsonl"
chunks = []
chunk_size = 800  # 800자 단위로 자르기
chunk_overlap = 100 # 문맥 유지를 위해 100자씩 겹침

with open(input_path, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        text = data['본문']
        
        # 텍스트를 일정 길이로 분할
        for i in range(0, len(text), chunk_size - chunk_overlap):
            chunk_text = text[i:i + chunk_size]
            chunks.append({
                "사업명": data['사업명'],
                "content": chunk_text,
                "metadata": {
                    "source": data['파일명'],
                    "chunk_id": i // (chunk_size - chunk_overlap)
                }
            })

# 2. 청킹 결과 저장
chunk_path = "/home/spai0630/workspace/data/chunks.jsonl"
with open(chunk_path, 'w', encoding='utf-8') as f:
    for chunk in chunks:
        f.write(json.dumps(chunk, ensure_ascii=False) + "\n")

print(f"✅ 총 {len(chunks)}개의 의미 단위(Chunk)가 생성되었습니다!")

✅ 총 656개의 의미 단위(Chunk)가 생성되었습니다!


In [15]:
import json
import os

input_file = "/home/spai0630/workspace/data/processed_pdf_only.jsonl"
output_file = "/home/spai0630/workspace/data/final_chunks.jsonl"

chunks = []
# RFP 문서는 보통 조항이 길기 때문에 500자 단위가 적당합니다.
CHUNK_SIZE = 500 
OVERLAP = 50

if os.path.exists(input_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            item = json.loads(line)
            text = item['본문']
            # 청킹 로직
            for i in range(0, len(text), CHUNK_SIZE - OVERLAP):
                chunk = text[i:i + CHUNK_SIZE]
                chunks.append({
                    "content": chunk,
                    "metadata": {"source": item['파일명'], "title": item['사업명']}
                })
    
    with open(output_file, 'w', encoding='utf-8') as f:
        for c in chunks:
            f.write(json.dumps(c, ensure_ascii=False) + "\n")
            
    print(f"✅ 4개 문서에서 총 {len(chunks)}개의 청크를 생성했습니다.")
else:
    print("❌ 전처리된 JSONL 파일을 찾을 수 없습니다.")

✅ 4개 문서에서 총 1021개의 청크를 생성했습니다.


In [16]:
import json
import random

chunk_path = "/home/spai0630/workspace/data/final_chunks.jsonl"

# 데이터 로드
with open(chunk_path, 'r', encoding='utf-8') as f:
    all_chunks = [json.loads(line) for line in f]

print(f"📦 전체 청크 수: {len(all_chunks)}개")
print("-" * 50)

# 랜덤하게 3개 추출해서 확인
sample_chunks = random.sample(all_chunks, 3)

for i, chunk in enumerate(sample_chunks):
    print(f"✨ 샘플 {i+1}")
    print(f"📂 출처: {chunk['metadata']['source']}")
    print(f"📌 사업명: {chunk['metadata']['title']}")
    print(f"📝 내용 요약 (앞 200자): \n{chunk['content'][:200]}...")
    print(f"📏 실제 길이: {len(chunk['content'])}자")
    print("-" * 50)

📦 전체 청크 수: 1021개
--------------------------------------------------
✨ 샘플 1
📂 출처: 서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf
📌 사업명: 2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용역
📝 내용 요약 (앞 200자): 
 등을 반입 및 반출 시 발주기관의 승인을 받아야 하며, 해당 가. 사업 수행을 위해 발주기관에서 용역업체에게 제공하는 내부 자료는 자료관리 대장을
장비에 유료 최신 백신을 설치하고 악성코드를 주기적으로 검사해야 한다. 작성하고, 인계자․인수자가 자필로 서명해야 한다.
다. 발주기관은 용역업체에게 제공한 사무실에 대해 수시로 보안점검을 할 수 있다. 나. ...
📏 실제 길이: 500자
--------------------------------------------------
✨ 샘플 2
📂 출처: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
📌 사업명: 차세대 포털·학사 정보시스템 구축사업 
📝 내용 요약 (앞 200자): 

사 업 명 사 업 기 간 계 약 금 액 발 주 처 비 고
※ 연도순으로 기재하며, 제안과제와 유사하거나 동일한 업무영역이나 사업형태에 관한
것만 기재한다. 단, 현재수행중인 사업은 비고란에 현재수행중임을 명시한다.
※ 하도급은 발주처가 승인한 경우에 한하여 작성하며 비고란에 원도급회사를 기재한다.
※ 공동도급계약일 경우에는 계약금액란에 제안사의 지분만을 ...
📏 실제 길이: 500자
--------------------------------------------------
✨ 샘플 3
📂 출처: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
📌 사업명: 2025년도 중이온가속기용 극저온시스템 운전 용역
📝 내용 요약 (앞 200자): 
영될 수 있도록,
중이온가속기용 극저온시스템 i) 극저온시스템을 모니터링 및 제어하고,
운전 업무 ii) 극저온시스템의 주요 수치를 기

글자수 기반 청킹 결과가 제법 좋다. 
글자 깨짐도 없고 출처와 사업명 메타 데이터 역시도 정확하게 나오고 있다.
하지만 그래도 중간에 문장이 끊기는 현상이 발생하고 있어서 
의미론적 청킹을 시도해보고 싶다

In [18]:
pip install langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [20]:
# 의미 분석 엔진과 실험적 기능 패키지를 함께 설치합니다.
!pip install sentence-transformers langchain-experimental

  Using cached setuptools-80.10.2-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 11.3 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 8.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 9.8 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 13.8 MB/s  0:00:52m0:00:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 34.1 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 16.2 MB/s  0:00:30m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 31.3 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 33.5 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 28.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 14.7 MB/s  0:

In [21]:
# 필요 패키지 설치: pip install langchain-experimental
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
import json

# 1. 임베딩 모델 로드 (의미 분석을 위해 필요)
print("🧠 의미 분석을 위한 모델 로드 중...")
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

# 2. 의미론적 청커 설정
# breakpoint_threshold_type은 문장 간의 차이를 어떻게 계산할지 결정합니다.
text_splitter = SemanticChunker(
    embeddings, 
    breakpoint_threshold_type="percentile" # 하위 %의 유사도를 기준으로 분할
)

# 3. 원본 데이터 로드 (JSONL에서 텍스트만 추출)
input_path = "/home/spai0630/workspace/data/processed_pdf_only.jsonl"
with open(input_path, 'r', encoding='utf-8') as f:
    raw_data = [json.loads(line) for line in f]

semantic_chunks = []
print("✂️ 의미론적 청킹 시작 (시간이 좀 걸립니다)...")

for item in raw_data:
    docs = text_splitter.create_documents([item['본문']])
    for i, doc in enumerate(docs):
        semantic_chunks.append({
            "content": doc.page_content,
            "metadata": {
                "source": item['파일명'],
                "title": item['사업명'],
                "chunk_id": i,
                "type": "semantic"
            }
        })

print(f"✅ 의미론적 청킹 완료! 생성된 청크 수: {len(semantic_chunks)}개")

# 샘플 확인 (첫 번째 것만)
if semantic_chunks:
    print("\n📝 의미론적 청크 샘플:")
    print(semantic_chunks[0]['content'][:300])

🧠 의미 분석을 위한 모델 로드 중...


/home/spai0630/workspace/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 199/199 [00:00<00:00, 641.23it/s, Materializing param=pooler.dense.weight]                               
RobertaModel LOAD REPORT from: jhgan/ko-sroberta-multitask
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✂️ 의미론적 청킹 시작 (시간이 좀 걸립니다)...
✅ 의미론적 청킹 완료! 생성된 청크 수: 141개

📝 의미론적 청크 샘플:
제 안 요 청 서
고려대학교
차세대 포털·학사 정보시스템 구축 사업
2024. 7. 01
※ 본 자료는 제안내용의 설명을 위한 배포자료로, 이외 목적으로 무단복제, 전달 및 사용하는 행위를 금함. -1- 목 차
Ⅰ. 사업개요 ·················································································································· 4
1. 사업 개요 ················································


길이 기반 청킹에서는 1021개 였던 청크가 의미 기반 청킹하니까 141개로 대폭 줄었습니다. 

길이 기반 처럼 내용이 중간에 맥락없이 잘리는 현상 역시 사라져 훨씬 자연스러운 흐름을 만들어낼 가능성이 높아 보입니다.
